In [1]:
import re
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
!pip install factor_analyzer

#### Loading in data 
after initial load, no need to run the immediate cell and instead, load acc_df from the pkl file.

In [ ]:
# need to obtain the tar file from lending club loan data on Kaggle
# filepath = './'
# folders = os.listdir(filepath)
# acc_df = pd.read_csv("./accepted_2007_to_2018Q4.csv.gz")
# acc_df.head()
# acc_df.to_pickle('acc_df_cached.pkl')

In [2]:
acc_df = pd.read_pickle('acc_df_cached.pkl') # read from current directory, this is faster

In [4]:
acc_df.columns[0:50]

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d'],
      dtype='object')

In [6]:
acc_df.columns[100:]

Index(['num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies',
       'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'revol_bal_joint',
       'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_flag', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_

In [62]:
acc_df.columns[50:100]

Index(['last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_

In [3]:
acc_df.shape
acc_df.head().T
acc_df["loan_status"].value_counts()

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

### Data Cleaning
Three categories of loan status results
* Fully Paid
* Late (grace period, 16-30, 31-120)
* Charged off / default

In [14]:
# Preprocessing certain columns
acc_df["pct_bc_inssats"] = 1 - acc_df["num_bc_sats"]/acc_df["num_bc_tl"] 
acc_df["pct_tl_evr_dlq"] = 1 - acc_df["pct_tl_nvr_dlq"] / 100
acc_df["percent_bc_gt_75"] = acc_df["percent_bc_gt_75"] / 100
acc_df["pct_acc_now_delinq"] = acc_df["acc_now_delinq"] / acc_df["total_acc"]

In [37]:

# different combinations of explanatory columns
# * watch out for high collinearity
explore_columns = {
    1: ["tax_liens", "collections_12_mths_ex_med", "pub_rec_bankruptcies", "num_accts_ever_120_pd"],
    2: ["tax_liens", "collections_12_mths_ex_med", "pub_rec_bankruptcies", "pct_tl_nvr_dlq", "num_accts_ever_120_pd"],
    3: ["tax_liens", "pub_rec_bankruptcies"],
    4: [],
    5: ["pct_tl_evr_dlq", "pct_bc_inssats", "tax_liens", "delinq_2yrs", "chargeoff_within_12_mths", "collections_12_mths_ex_med", "acc_now_delinq", "percent_bc_gt_75", "pub_rec_bankruptcies", "pub_rec", "num_accts_ever_120_pd", "dti"],
    6: ["pct_tl_evr_dlq", "pct_bc_inssats", "delinq_2yrs", "chargeoff_within_12_mths", "pub_rec_bankruptcies", "num_accts_ever_120_pd", "pct_acc_now_delinq"]
}

# list of relevant columns
info = ["dti_joint", "verification_status_joint", "grade", "sub_grade"]
dependent = ["loan_status"]
explanatory = explore_columns[6]

In [38]:
# filter for relevant columns
filtered_df = acc_df[info + dependent + explanatory]

# data cleaning - given that NAs make up a small portion of the overall data, we drop NAs
print(f"count before drop na: {len(filtered_df)}, count after drop na: {len(filtered_df.dropna(subset=explanatory+dependent) )}")
filtered_df = filtered_df.dropna(subset=explanatory+dependent) # after dropping, 2260701 -> 2106856

# split based on loan status
late_df = filtered_df[filtered_df['loan_status'].isin(["Late (16-30 days)", "Late (31-120 days)", "In Grace Period"])]
defaulted_df = filtered_df[filtered_df['loan_status'].isin(["Default", "Charged Off"])]
paid_df = filtered_df[filtered_df['loan_status'] == "Fully Paid"]

# map loan status to an indicator variable TODO - decide how to categorize lates
value_map = {'Fully Paid': 0, 'Default': 1, 'Charged Off': 1, "Late (16-30 days)": 0, "Late (31-120 days)": 0, "In Grace Period": 0}
filtered_df["default_ind"] = filtered_df['loan_status'].map(value_map)

count before drop na: 2260701, count after drop na: 2184536


In [39]:
filtered_df.head()

,dti_joint,verification_status_joint,grade,sub_grade,loan_status,pct_tl_evr_dlq,pct_bc_inssats,delinq_2yrs,chargeoff_within_12_mths,pub_rec_bankruptcies,num_accts_ever_120_pd,pct_acc_now_delinq,default_ind
0,NaN,NaN,C,C4,Fully Paid,0.231,0.600000,0.0,0.0,0.0,2.0,0.0,0.0
1,NaN,NaN,C,C1,Fully Paid,0.026,0.235294,1.0,0.0,0.0,0.0,0.0,0.0
2,13.85,Not Verified,B,B4,Fully Paid,0.000,0.500000,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,C,C5,Current,0.000,0.200000,0.0,0.0,0.0,0.0,0.0,NaN
4,NaN,NaN,F,F1,Fully Paid,0.034,0.444444,1.0,0.0,0.0,0.0,0.0,0.0


### Latent Variables

columns of interest (explantory variates):

a factor model resembles a series of regressions to predict the observed variables from the unobserved.

Each of our response variables X is predicted as a linear function of the unobserved common factors

we want one unobserved common factor, responsibility (fr)

mean 0 and variance 1

In [40]:
# covariance matrix
filtered_df[explanatory].corr()

,pct_tl_evr_dlq,pct_bc_inssats,delinq_2yrs,chargeoff_within_12_mths,pub_rec_bankruptcies,num_accts_ever_120_pd,pct_acc_now_delinq
pct_tl_evr_dlq,1.000000,0.146455,0.442554,0.083019,-0.050604,0.578883,0.046928
pct_bc_inssats,0.146455,1.000000,0.104917,0.043815,0.059441,0.164600,0.019492
delinq_2yrs,0.442554,0.104917,1.000000,0.140533,-0.051391,0.218243,0.099081
chargeoff_within_12_mths,0.083019,0.043815,0.140533,1.000000,-0.011715,0.119129,0.028722
pub_rec_bankruptcies,-0.050604,0.059441,-0.051391,-0.011715,1.000000,-0.032076,-0.009033
num_accts_ever_120_pd,0.578883,0.164600,0.218243,0.119129,-0.032076,1.000000,0.014851
pct_acc_now_delinq,0.046928,0.019492,0.099081,0.028722,-0.009033,0.014851,1.000000


In [104]:
# # for linear regression - use this code if we are predicting continuous response variates (?)
# from sklearn import linear_model
# regr = linear_model.LinearRegression()

# import statsmodels.api as sm

# subset_expl = ["pub_rec_bankruptcies","tax_liens"]
# X = filtered_df[subset_expl]
# y = filtered_df[["default_likelihood"]]
# X = sm.add_constant(X) # add intercept term

# # Define and fit the model
# model = sm.OLS(y, X)
# results = model.fit()
# print(results.summary())

### One Factor Confirmatory Factor Analysis
determine if our selected columns of interests do share an underlying factor

https://www.datacamp.com/tutorial/introduction-factor-analysis

In [18]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

In [41]:
df = filtered_df[explanatory]

In [42]:
# perform an adequacy test
chi_square_value,p_value=calculate_bartlett_sphericity(df)
chi_square_value, p_value

# observe that there is a very small p-value, thus this test passes.

(1554455.8792255882, 0.0)

In [43]:
kmo_all,kmo_model=calculate_kmo(df)
kmo_model
# value of 50 is not ideal, but maybe we can go with it

/home/ye/.pyenv/versions/3.10.14/envs/abi-op-service-env/lib/python3.10/site-packages/factor_analyzer/utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


0.5816380917665808

In [44]:
fa = FactorAnalyzer(n_factors=1,rotation='varimax')
fa.fit(df)
factor_df=pd.DataFrame(fa.loadings_,index=df.columns)
print(factor_df)

                                 0
pct_tl_evr_dlq           -0.932335
pct_bc_inssats           -0.195933
delinq_2yrs              -0.457469
chargeoff_within_12_mths -0.150994
pub_rec_bankruptcies      0.053554
num_accts_ever_120_pd    -0.603487
pct_acc_now_delinq       -0.070801


/home/ye/.pyenv/versions/3.10.14/envs/abi-op-service-env/lib/python3.10/site-packages/factor_analyzer/factor_analyzer.py:663: UserWarning: No rotation will be performed when the number of factors equals 1.
  warnings.warn(


In [45]:
fa.get_factor_variance()

(array([1.51179281]), array([0.2159704]), array([0.2159704]))

In [46]:
print(f"total explained variance by factor: {sum(sum(fa.get_factor_variance()))}")

total explained variance by factor: 1.9437336184841518


In [47]:
fa.loadings_

array([[-0.93233505],
       [-0.19593258],
       [-0.45746858],
       [-0.15099395],
       [ 0.05355386],
       [-0.60348743],
       [-0.07080127]])

In [48]:
# get factor scores (documentation: https://readthedocs.org/projects/factor-analyzer/downloads/pdf/latest/)
fa.transform(df.values)

array([[-1.7441522 ],
       [ 0.3257965 ],
       [ 0.57112217],
       ...,
       [-2.54186954],
       [ 0.13798601],
       [-1.42561821]])

### Logistic Regression

In [56]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


### Train Model

In [57]:
# generate irresponsibility measure
pca = PCA(n_components=1)
principal_component = pca.fit_transform(filtered_df[explanatory])
scaler = MinMaxScaler(feature_range=(0, 1))
irresponsibility_measure = scaler.fit_transform(principal_component)

In [58]:
print(f"count before drop na: {len(filtered_df)}, ")
print(f"count after drop na: {len(filtered_df.dropna(subset=['default_ind']))}") # this na drop is a bit high

filtered_df = filtered_df.dropna(subset=["default_ind"]) 
x = np.array(irresponsibility_measure)
y = np.array(filtered_df["default_ind"])          # response
x = sm.add_constant(x)  

# split test and train set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)
model = sm.Logit(y_train, x_train).fit()

count before drop na: 1308894, 
count after drop na: 1308894
Optimization terminated successfully.
         Current function value: 0.495937
         Iterations 5


In [59]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:               916225
Model:                          Logit   Df Residuals:                   916223
Method:                           MLE   Df Model:                            1
Date:                Sat, 26 Oct 2024   Pseudo R-squ.:               0.0001528
Time:                        20:47:40   Log-Likelihood:            -4.5439e+05
converged:                       True   LL-Null:                   -4.5446e+05
Covariance Type:            nonrobust   LLR p-value:                 4.744e-32
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4209      0.003   -486.950      0.000      -1.427      -1.415
x1             1.1071      0.093     11.944      0.000       0.925       1.289
==============================================================================
"""

### Test Model

In [64]:
yhat = model.predict(x_test)
predction = list(map(round, yhat))
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predction)
print("Confusion Matrix: \n", cm)

print('Test accuracy = ', accuracy_score(y_test, predction))

Confusion Matrix: 
 [[315517      0]
 [ 77152      0]]
Test accuracy =  0.8035189943693034
